In [ ]:
#### 引入套件 
import os
from sqlalchemy import create_engine
from sqlalchemy_utils.functions import database_exists, create_database
from sqlalchemy.types import NVARCHAR, Float, Integer
import cymysql

In [ ]:
#### DB 參數設置
username = 'root'
password = 'your_password'
host = 'localhost'
port = '3306'
new_db_name = 'Project_Test'
table_name = 'table_a'

In [ ]:
#### 將 dataframe 的資料型態轉成 DB 格式
dtypedict = {
    'str': NVARCHAR(length=255),
    'int': Integer(),
    'float': Float()
}

In [ ]:
#### 若資料庫未建立則建立並連接
#### 若資料庫已建立則連接
def GetEngine(username, password, host, port, new_db_name):
    '''
    create new db if not exist
    mysql+cymysql：根據對接的資料庫系統而定（若有中文需加 cymysql）
    echo: 返還 SQL碼
    '''
    engine = create_engine(f'mysql+cymysql://{username}:{password}@{host}:{port}/{new_db_name}',
                           echo=True, encoding='utf8') 
    if database_exists(engine.url):
        print(f'{new_db_name} exists.')
    else:
        create_database(engine.url)
        print(f'{new_db_name} is created.')
    return engine

In [ ]:
#### dump pandas dataframe to table
#### SQLAlchemy types:https://docs.sqlalchemy.org/en/14/core/type_basics.html
def DumpCsvToDB(engine, data_path, table_name):
    # read csv
    data = pd.read_csv(data_path)
    
    # dump pandas dataframe to table
    # if data exists then replace
    data.to_sql(con=engine, name=table_name,
                if_exists='replace',
                dtype=dtypedict)

In [ ]:
#### 執行主要程式
engine = GetEngine(username, password, host, port, new_db_name)
DumpCsvToDB(engine, data_path, table_name)

#### 讀取資料庫表格
df = pd.read_sql_table(table_name, engine)